In [1]:
from IPython.display import Markdown as md

### change to reflect your notebook
_nb_loc = "09_deploying/09f_explain.ipynb"
_nb_title = "Explain predictions"

### no need to change any of this
_nb_safeloc = _nb_loc.replace('/', '%2F')
md("""
<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://console.cloud.google.com/ai-platform/notebooks/deploy-notebook?name={1}&url=https%3A%2F%2Fgithub.com%2FGoogleCloudPlatform%2Fpractical-ml-vision-book%2Fblob%2Fmaster%2F{2}&download_url=https%3A%2F%2Fgithub.com%2FGoogleCloudPlatform%2Fpractical-ml-vision-book%2Fraw%2Fmaster%2F{2}">
    <img src="https://raw.githubusercontent.com/GoogleCloudPlatform/practical-ml-vision-book/master/logo-cloud.png"/> Run in AI Platform Notebook</a>
  </td>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/GoogleCloudPlatform/practical-ml-vision-book/blob/master/{0}">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/GoogleCloudPlatform/practical-ml-vision-book/blob/master/{0}">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://raw.githubusercontent.com/GoogleCloudPlatform/practical-ml-vision-book/master/{0}">
    <img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>
""".format(_nb_loc, _nb_title, _nb_safeloc))


<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://console.cloud.google.com/ai-platform/notebooks/deploy-notebook?name=Explain predictions&url=https%3A%2F%2Fgithub.com%2FGoogleCloudPlatform%2Fpractical-ml-vision-book%2Fblob%2Fmaster%2F09_deploying%2F09f_explain.ipynb&download_url=https%3A%2F%2Fgithub.com%2FGoogleCloudPlatform%2Fpractical-ml-vision-book%2Fraw%2Fmaster%2F09_deploying%2F09f_explain.ipynb">
    <img src="https://raw.githubusercontent.com/GoogleCloudPlatform/practical-ml-vision-book/master/logo-cloud.png"/> Run in AI Platform Notebook</a>
  </td>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/GoogleCloudPlatform/practical-ml-vision-book/blob/master/09_deploying/09f_explain.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/GoogleCloudPlatform/practical-ml-vision-book/blob/master/09_deploying/09f_explain.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://raw.githubusercontent.com/GoogleCloudPlatform/practical-ml-vision-book/master/09_deploying/09f_explain.ipynb">
    <img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>


# Explain predictions

In this notebook, we explain model predictions

## Enable GPU and set up helper functions

This notebook and pretty much every other notebook in this repository
will run faster if you are using a GPU.
On Colab:
- Navigate to Edit→Notebook Settings
- Select GPU from the Hardware Accelerator drop-down

On Cloud AI Platform Notebooks:
- Navigate to https://console.cloud.google.com/ai-platform/notebooks
- Create an instance with a GPU or select your instance and add a GPU

Next, we'll confirm that we can connect to the GPU with tensorflow:

In [ ]:
import tensorflow as tf
print('TensorFlow version' + tf.version.VERSION)
print('Built with GPU support? ' + ('Yes!' if tf.test.is_built_with_cuda() else 'Noooo!'))
print('There are {} GPUs'.format(len(tf.config.experimental.list_physical_devices("GPU"))))
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

## Model with preprocess signature

We start from the checkpoint

In [1]:
import tensorflow as tf
import os, shutil

MODEL_LOCATION='export/flowers_model3'  # will be created

In [ ]:
# load from checkpoint and export a model that has desired signature
CHECK_POINT_DIR='gs://practical-ml-vision-book/flowers_5_trained/chkpts'
model = tf.keras.models.load_model(CHECK_POINT_DIR)

In [4]:
IMG_HEIGHT = 345
IMG_WIDTH = 345
IMG_CHANNELS = 3
CLASS_NAMES = 'daisy dandelion roses sunflowers tulips'.split()

def preprocess(filename):
    img_bytes = tf.io.read_file(filename)
    img = tf.image.decode_jpeg(img_bytes, channels=IMG_CHANNELS)
    img = tf.image.convert_image_dtype(img, tf.float32)
    return tf.image.resize_with_pad(img, IMG_HEIGHT, IMG_WIDTH)

@tf.function(input_signature=[tf.TensorSpec([None, IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS], dtype=tf.float32)])
def xai_model(input_images):
    batch_pred = model(input_images) # same as model.predict()
    top_prob = tf.math.reduce_max(batch_pred, axis=[1])
    pred_label_index = tf.math.argmax(batch_pred, axis=1)
    pred_label = tf.gather(tf.convert_to_tensor(CLASS_NAMES), pred_label_index)
    return {
        'probability': top_prob,
        'flower_type_int': pred_label_index,
        'flower_type_str': pred_label
    }

@tf.function(input_signature=[tf.TensorSpec([None,], dtype=tf.string)])
def xai_preprocess(filenames):
    input_images = tf.map_fn(
        preprocess,
        filenames,
        fn_output_signature=tf.float32
    )
    return {
        # match the signature of xai_model
        'input_images': input_images
    }

@tf.function(input_signature=[tf.TensorSpec([None,], dtype=tf.string)])
def predict_filename(filenames):
    preproc_output = xai_preprocess(filenames)
    return xai_model(preproc_output['input_images'])

shutil.rmtree('export', ignore_errors=True)
os.mkdir('export')
model.save(MODEL_LOCATION,
          signatures={
              'serving_default': predict_filename,
              'xai_preprocess': xai_preprocess, # should be exactly what's supplied to model
              'xai_model': xai_model # call the model
          })

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: export/flowers_model3/assets


In [24]:
!saved_model_cli show --dir {MODEL_LOCATION} --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['filenames'] tensor_info:
        dtype: DT_STRING
        shape: (-1)
        name: serving_default_filenames:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['flower_type_int'] tensor_info:
        dtype: DT_INT64
        shape: (-1)
        name: StatefulPartitionedCall:0
    outputs['flower_type_str'] tensor_info:
        dtype: DT_STRING
        shape: (-1)
        name: StatefulPartitionedCall:1
    outputs['probability'] tensor_info:
        d

## Try out the new signatures

In [5]:
serving_fn = tf.keras.models.load_model(MODEL_LOCATION).signatures['serving_default']
filenames = [
    'gs://cloud-ml-data/img/flower_photos/dandelion/9818247_e2eac18894.jpg',
    'gs://cloud-ml-data/img/flower_photos/dandelion/9853885425_4a82356f1d_m.jpg',
    'gs://cloud-ml-data/img/flower_photos/daisy/9299302012_958c70564c_n.jpg',
    'gs://cloud-ml-data/img/flower_photos/tulips/8733586143_3139db6e9e_n.jpg',
    'gs://cloud-ml-data/img/flower_photos/tulips/8713397358_0505cc0176_n.jpg'
]
pred = serving_fn(tf.convert_to_tensor(filenames))
print(pred)

{'flower_type_int': <tf.Tensor: shape=(5,), dtype=int64, numpy=array([1, 1, 0, 4, 4])>, 'flower_type_str': <tf.Tensor: shape=(5,), dtype=string, numpy=
array([b'dandelion', b'dandelion', b'daisy', b'tulips', b'tulips'],
      dtype=object)>, 'probability': <tf.Tensor: shape=(5,), dtype=float32, numpy=
array([0.39833778, 0.9999609 , 0.9947188 , 0.95900667, 0.94177294],
      dtype=float32)>}


In [ ]:
serving_fn = tf.keras.models.load_model(MODEL_LOCATION).signatures['xai_preprocess']
filenames = [
    'gs://cloud-ml-data/img/flower_photos/dandelion/9818247_e2eac18894.jpg',
]
pred = serving_fn(tf.convert_to_tensor(filenames))
print(pred)

## Create explanation metadata

We can use a Python API to create explanations.json or we can hand-create it.

As a baseline, we'll use a random image (we could also use all zeros or all ones)

In [26]:
from explainable_ai_sdk.metadata.tf.v2 import SavedModelMetadataBuilder
import numpy as np

export_path=MODEL_LOCATION  # save with model
# We want to explain 'xai_model' signature.
builder = SavedModelMetadataBuilder(
    export_path,
    signature_name='xai_model',
    outputs_to_explain=['probability'])
random_baseline = np.random.rand(IMG_HEIGHT, IMG_WIDTH, 3)
builder.set_image_metadata(
    'input_images',
    input_baselines=[random_baseline.tolist()])
builder.save_metadata(export_path)

In [27]:
!head -20 {MODEL_LOCATION}/explanation_metadata.json

{
  "outputs": {
    "probability": {
      "output_tensor_name": "probability"
    }
  },
  "inputs": {
    "input_images": {
      "input_tensor_name": "input_images",
      "encoding": "identity",
      "modality": "image",
      "input_baselines": [
        [
          [
            [
              0.9785163327444246,
              0.9643898229957655,
              0.420070782929297
            ],
            [


## Deploy explanation model to Cloud AI Platform

In [6]:
!gsutil cp -r {MODEL_LOCATION} gs://ai-analytics-solutions/tmp/flowers_explanations

Copying file://export/flowers_model3/explanation_metadata.json [Content-Type=application/json]...
Copying file://export/flowers_model3/saved_model.pb [Content-Type=application/octet-stream]...
Copying file://export/flowers_model3/variables/variables.data-00000-of-00001 [Content-Type=application/octet-stream]...
Copying file://export/flowers_model3/variables/variables.index [Content-Type=application/octet-stream]...
- [4 files][ 25.6 MiB/ 25.6 MiB]                                                
Operation completed over 4 objects/25.6 MiB.                                     


In [7]:
!./caip_deploy_regional.sh --model_location {MODEL_LOCATION} #gs://ai-analytics-solutions/tmp/flowers_explanations

Deploying flowers_regional:ig from gs://ai-analytics-solutions/tmp/flowers_explanations
Using endpoint [https://us-central1-ml.googleapis.com/]
The model named flowers_regional already exists.
Using endpoint [https://us-central1-ml.googleapis.com/]
Deleting already the existing model flowers_regional:ig ... 
Using endpoint [https://us-central1-ml.googleapis.com/]
Deleting version [ig]......done.                                               
Please run this script again if you don't see a Creating message ... 
Creating flowers_regional:ig --explanation-method integrated-gradients --num-integral-steps 25
Using endpoint [https://us-central1-ml.googleapis.com/]
Explanations reflect patterns in your model, but don't necessarily reveal fundamental relationships about your data population. See https://cloud.google.com/ml-engine/docs/ai-explanations/limitations for more information.


In [17]:
%%writefile request.json
{
    "instances": [
        {
            "filenames": "gs://cloud-ml-data/img/flower_photos/dandelion/9818247_e2eac18894.jpg"
        },
        {
            "filenames": "gs://cloud-ml-data/img/flower_photos/dandelion/9853885425_4a82356f1d_m.jpg"
        },
        {
            "filenames": "gs://cloud-ml-data/img/flower_photos/daisy/9299302012_958c70564c_n.jpg"
        },
        {
            "filenames": "gs://cloud-ml-data/img/flower_photos/tulips/8733586143_3139db6e9e_n.jpg"
        },
        {
            "filenames": "gs://cloud-ml-data/img/flower_photos/tulips/8713397358_0505cc0176_n.jpg"
        }
    ]
}

Overwriting request.json


In [20]:
!gcloud ai-platform predict --region=us-central1 --model=flowers_regional --version=ig --json-request=request.json

Using endpoint [https://us-central1-ml.googleapis.com/]
ERROR: (gcloud.ai-platform.predict) HTTP request failed. Response: {
  "error": {
    "code": 404,
    "message": "Field: name Error: Online prediction is unavailable for this version. Please verify that CreateVersion has completed successfully.",
    "status": "NOT_FOUND",
    "details": [
      {
        "@type": "type.googleapis.com/google.rpc.BadRequest",
        "fieldViolations": [
          {
            "field": "name",
            "description": "Online prediction is unavailable for this version. Please verify that CreateVersion has completed successfully."
          }
        ]
      }
    ]
  }
}



## License
Copyright 2020 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.